In [3]:
import torch

## load checkpoint

In [4]:
pathe = '/home/rozenberszki/project/wsnsl/output/Own/room1_small/00059.tar'
traj = '/home/rozenberszki/project/wsnsl/Datasets/Replica/room1/traj.txt'

In [11]:
d = torch.load(pathe)

: 

In [6]:
T_e = d['estimate_c2w_list']
#T_e[:,1:3]*= -1

In [7]:
T_e[16]

tensor([[-3.0114e-01,  1.1277e-01,  9.4689e-01, -2.1420e+00],
        [ 9.5358e-01,  3.4882e-02,  2.9912e-01, -4.0176e-01],
        [ 7.0141e-04,  9.9301e-01, -1.1804e-01,  2.1823e-01],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]])

In [8]:
import numpy as np
T_wc = np.loadtxt(traj).reshape(-1,4,4)
T_wc[:,1:3]*= -1


In [9]:
np.round(T_e[5].numpy(),3)

array([[-0.174,  0.227,  0.958, -2.211],
       [ 0.985,  0.044,  0.168, -0.463],
       [-0.004,  0.973, -0.231,  0.171],
       [ 0.   ,  0.   ,  0.   ,  1.   ]], dtype=float32)

In [10]:
np.round(T_wc[10],2), np.round(T_e[10].numpy(),2),np.round(T_wc[10]- T_e[10].numpy(),3)

(array([[-0.24, -0.16, -0.96, -2.18],
        [-0.97,  0.04,  0.23,  0.43],
        [-0.  ,  0.99, -0.17, -0.21],
        [ 0.  ,  0.  ,  0.  ,  1.  ]]),
 array([[-0.24,  0.17,  0.96, -2.18],
        [ 0.97,  0.04,  0.23, -0.44],
        [-0.  ,  0.98, -0.18,  0.19],
        [ 0.  ,  0.  ,  0.  ,  1.  ]], dtype=float32),
 array([[-2.000e-03, -3.370e-01, -1.914e+00, -0.000e+00],
        [-1.943e+00, -3.000e-03,  2.000e-03,  8.660e-01],
        [ 1.000e-03,  2.000e-03,  9.000e-03, -4.040e-01],
        [ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00]]))

In [40]:
m = np.ones((10,4,4),int)
s = np.ones((4,4), int)
s[[0,0,1,1,2], [1,2,0,3,3]] *=-1
m*s

array([[[ 1, -1, -1,  1],
        [-1,  1,  1, -1],
        [ 1,  1,  1, -1],
        [ 1,  1,  1,  1]],

       [[ 1, -1, -1,  1],
        [-1,  1,  1, -1],
        [ 1,  1,  1, -1],
        [ 1,  1,  1,  1]],

       [[ 1, -1, -1,  1],
        [-1,  1,  1, -1],
        [ 1,  1,  1, -1],
        [ 1,  1,  1,  1]],

       [[ 1, -1, -1,  1],
        [-1,  1,  1, -1],
        [ 1,  1,  1, -1],
        [ 1,  1,  1,  1]],

       [[ 1, -1, -1,  1],
        [-1,  1,  1, -1],
        [ 1,  1,  1, -1],
        [ 1,  1,  1,  1]],

       [[ 1, -1, -1,  1],
        [-1,  1,  1, -1],
        [ 1,  1,  1, -1],
        [ 1,  1,  1,  1]],

       [[ 1, -1, -1,  1],
        [-1,  1,  1, -1],
        [ 1,  1,  1, -1],
        [ 1,  1,  1,  1]],

       [[ 1, -1, -1,  1],
        [-1,  1,  1, -1],
        [ 1,  1,  1, -1],
        [ 1,  1,  1,  1]],

       [[ 1, -1, -1,  1],
        [-1,  1,  1, -1],
        [ 1,  1,  1, -1],
        [ 1,  1,  1,  1]],

       [[ 1, -1, -1,  1],
        [-1,  1,  1,

## check decoder update

In [12]:
from src.conv_onet.models.decoder import DenseLayer
from src import config

In [13]:
cfg = config.load_config(  # J:changed it to use our config file including semantics
        '/home/koerner/Project/wsnsl/configs/Own/room1.yaml', "configs/nice_slam_sem.yaml"
    )

In [14]:

model = config.get_model(cfg, nice=True)


In [15]:
model

NICE(
  (middle_decoder): MLP(
    (fc_c): ModuleList(
      (0): Linear(in_features=32, out_features=32, bias=True)
      (1): Linear(in_features=32, out_features=32, bias=True)
      (2): Linear(in_features=32, out_features=32, bias=True)
      (3): Linear(in_features=32, out_features=32, bias=True)
      (4): Linear(in_features=32, out_features=32, bias=True)
    )
    (embedder): GaussianFourierFeatureTransform()
    (pts_linears): ModuleList(
      (0): DenseLayer(in_features=93, out_features=32, bias=True)
      (1): DenseLayer(in_features=32, out_features=32, bias=True)
      (2): DenseLayer(in_features=32, out_features=32, bias=True)
      (3): DenseLayer(in_features=125, out_features=32, bias=True)
      (4): DenseLayer(in_features=32, out_features=32, bias=True)
    )
    (output_linear): DenseLayer(in_features=32, out_features=1, bias=True)
  )
  (fine_decoder): MLP(
    (fc_c): ModuleList(
      (0): Linear(in_features=64, out_features=32, bias=True)
      (1): Linear(in_fe

In [10]:
model.semantic_decoder.output_linear = DenseLayer(in_dim=32, out_dim=100, activation='linear')

In [11]:
model

NICE(
  (middle_decoder): MLP(
    (fc_c): ModuleList(
      (0): Linear(in_features=32, out_features=32, bias=True)
      (1): Linear(in_features=32, out_features=32, bias=True)
      (2): Linear(in_features=32, out_features=32, bias=True)
      (3): Linear(in_features=32, out_features=32, bias=True)
      (4): Linear(in_features=32, out_features=32, bias=True)
    )
    (embedder): GaussianFourierFeatureTransform()
    (pts_linears): ModuleList(
      (0): DenseLayer(in_features=93, out_features=32, bias=True)
      (1): DenseLayer(in_features=32, out_features=32, bias=True)
      (2): DenseLayer(in_features=32, out_features=32, bias=True)
      (3): DenseLayer(in_features=125, out_features=32, bias=True)
      (4): DenseLayer(in_features=32, out_features=32, bias=True)
    )
    (output_linear): DenseLayer(in_features=32, out_features=1, bias=True)
  )
  (fine_decoder): MLP(
    (fc_c): ModuleList(
      (0): Linear(in_features=64, out_features=32, bias=True)
      (1): Linear(in_fe